`tqdm` 支持开箱即用的功能，意味着它可以直接在一个可循环的对象外层进行包裹，达到显示进度条的效果

`tqdm` 中的 `trange` 可以替代原来的 `range` 函数，达到显示进度条的效果

In [3]:
from tqdm import tqdm, trange
import time

for i in trange(100):
    time.sleep(0.05)

100%|██████████| 100/100 [00:05<00:00, 19.74it/s]


In [19]:
import random

lst = list(random.randint(0, 100) for _ in range(100))

for i in tqdm(lst):
    time.sleep(0.05)

100%|██████████| 100/100 [00:05<00:00, 19.72it/s]


`tqdm` 可以实例化一个 bar 对象，然后我们手动在其中进行进度条的更新
- 剩余时间是按照 total 和平均处理速度计算的，可能偏大

In [15]:
# 如果不确定总体长度可以写一个估值
bar = tqdm(total=120, desc="Test bar obj")

for i in range(100):
    time.sleep(0.05)
    bar.update(1)

# 消除进度条痕迹，避免影响后续输出
bar.close()

Test bar obj:  83%|████████▎ | 100/120 [00:05<00:01, 19.73it/s]


In [14]:
# 手动指定迭代单位
bar = tqdm(total=100, desc="Test bar obj", unit="words")

for i in range(100):
    time.sleep(0.05)
    bar.update(1)

# 消除进度条痕迹，避免影响后续输出
bar.close()

Test bar obj: 100%|██████████| 100/100 [00:05<00:00, 19.73words/s]


可以在运行过程中动态调整显示
- `set_description`: 调整描述
- `set_postfix`: 调整后缀显示内容

In [24]:
# 动态调整进度条 desc
name = ["zzzongzii", "landon", "SHU", "tsinghua"]

bar = tqdm(name, unit="people")

for p in bar:
    time.sleep(0.5)
    bar.set_description(f"Processing {p}")
    print()  # 让 tqdm 无法回到之前行，达到保留进度条的效果

Processing zzzongzii:  25%|██▌       | 1/4 [00:00<00:01,  2.00people/s]

Processing landon:  50%|█████     | 2/4 [00:01<00:01,  1.99people/s]   

Processing SHU:  75%|███████▌  | 3/4 [00:01<00:00,  1.99people/s]   

Processing tsinghua: 100%|██████████| 4/4 [00:02<00:00,  1.99people/s]

In [32]:
# 动态调整进度条 desc
name = ["zzzongzii", "landon", "SHU", "tsinghua"]

bar = tqdm(name, unit="people")

for p in bar:
    time.sleep(0.5)
    bar.set_description(f"Processing {p}")
    bar.set_postfix({"len": len(p)})
    print()  # 让 tqdm 无法回到之前行，达到保留进度条的效果

Processing zzzongzii:  25%|██▌       | 1/4 [00:00<00:01,  1.99people/s, len=9]

Processing landon:  50%|█████     | 2/4 [00:01<00:01,  1.99people/s, len=6]   

Processing SHU:  75%|███████▌  | 3/4 [00:01<00:00,  1.99people/s, len=3]   

Processing tsinghua: 100%|██████████| 4/4 [00:02<00:00,  1.99people/s, len=8]

可以手动控制 `tqdm` 的占位列数，以防止列数超过终端的显示宽度，导致 tqdm 不得不换行输出，使得显示混乱

In [28]:
for i in tqdm(range(99999), ncols=50):
    pass

100%|██| 99999/99999 [00:00<00:00, 4970564.88it/s]


In [29]:
for i in tqdm(range(99999), ncols=20):
    pass

100%|█| 99999/99999 


In [30]:
for i in tqdm(range(99999), ncols=4):
    pass

100%


`disable` 参数可以控制我们在调试代码的时候禁用以保证输出干净

In [31]:
debug_mode = True

for i in tqdm(range(1000), disable=debug_mode):
    pass

嵌套使用的 `tqdm`  

In [36]:
# 自动检测所处环境，使用对应样式

from tqdm.auto import tqdm

# 最顶层有一个父进度条
for i in tqdm(range(10)):
    time.sleep(0.05)
    for j in tqdm(range(10)):
        time.sleep(0.02)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

`tqdm` 可以和 `pandas` 一起合作，确保处理数据的过程也有进度条

In [38]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import time


def simulate_heavy_work(x: float) -> float:
    """
    模拟一个耗时的计算任务。

    Args:
        x (float): 输入的数值。

    Returns:
        float: 计算后的结果。
    """
    # 模拟耗时操作
    time.sleep(0.02)
    return x**2


def main():
    # 1. 注册 tqdm 到 pandas
    # 这步是关键，它会为 DataFrame 和 Series 增加 progress_apply 方法
    tqdm.pandas(desc="正在处理数据")

    # 2. 生成一个模拟数据集
    print("正在创建数据集...")
    df = pd.DataFrame(
        {
            "value": np.random.randn(500),
            "group": np.random.choice(["A", "B", "C", "D"], 500),
        }
    )

    # 3. 使用 progress_apply 替代 apply
    print("开始执行耗时任务：")
    df["result"] = df["value"].progress_apply(simulate_heavy_work)

    # 4. 在 groupby 操作中使用
    print("\n开始执行分组耗时任务：")
    grouped_result = df.groupby("group")["value"].progress_apply(
        lambda x: simulate_heavy_work(x.mean())
    )

    print("\n处理完成。")
    print(grouped_result)


if __name__ == "__main__":
    main()

正在创建数据集...
开始执行耗时任务：


正在处理数据:   0%|          | 0/500 [00:00<?, ?it/s]


开始执行分组耗时任务：


正在处理数据:   0%|          | 0/4 [00:00<?, ?it/s]


处理完成。
group
A    0.006233
B    0.000166
C    0.000649
D    0.001776
Name: value, dtype: float64
